In [1]:
import gensim
import numpy as np
import nltk
import re
from gensim.models import Word2Vec

In [2]:
# model = Word2Vec(corpus_file='tok_subtitles_data', vector_size=100, window=5, min_count=3, workers=4)
# print("Model built.")
# model.save("word2vec.model")

In [3]:
model = Word2Vec.load("word2vec.model")

In [4]:
decades = ["1930s", "1940s", "1950s", "1960s", "1970s", "1980s", "1990s", "2000s", "2010s"]

In [5]:
allmovies = []
current = []
movie_indices = []

count = 0
with open('tok_movie_data', 'r') as tok_movies, open('subtitles_data', 'r') as subs:
    subs.readline()
    while True:
        sub_line = subs.readline()
        line = tok_movies.readline()
        if not (line or sub_line):
            allmovies.append(current)
            break
        match = re.search(r"^\*\*\*[0-9]+\,[0-9]+\,[0-9]+\*\*\*$", sub_line)
        if match:
            allmovies.append(current)
            current = []
            count += 1
            movie_indices.append(count)
            sub_line = subs.readline()
            continue
        current.append(line)
        count += 1
        if (count%100000) == 0:
            print(count)

100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
2100000
2200000
2300000
2400000
2500000
2600000
2700000
2800000
2900000
3000000
3100000
3200000
3300000
3400000
3500000
3600000
3700000
3800000
3900000
4000000
4100000
4200000
4300000
4400000
4500000
4600000
4700000
4800000
4900000
5000000
5100000
5200000
5300000
5400000
5500000
5600000
5700000
5800000
5900000
6000000
6100000
6200000
6300000
6400000
6500000
6600000
6700000
6800000
6900000
7000000
7100000
7200000
7300000
7400000
7500000
7600000
7700000
7800000
7900000
8000000
8100000
8200000
8300000
8400000
8500000
8600000
8700000
8800000
8900000
9000000
9100000
9200000
9300000
9400000
9500000
9600000
9700000
9800000
9900000
10000000
10100000
10200000
10300000
10400000
10500000
10600000
10700000
10800000
10900000
11000000
11100000
11200000
11300000
11400000
11500000
11600000
11700000
11800000
11900000
12000000
12100000
12200000
12300000
1

In [6]:
print(len(allmovies))
print(allmovies[1][0])

13430
Thirty thousand . 



In [7]:
import pandas as pd

metadata = pd.read_excel('movie_metadata.xlsx', sheet_name='Sheet1')
df = pd.DataFrame(metadata, columns=['textID','title','year','decade','genre'])

movie_ids = df['textID'].tolist()
movie_decades = df['decade'].tolist()
movie_titles = df['title'].tolist()
movie_genres = df['genre'].tolist()
movie_years = df['year'].tolist()

In [8]:
import random

sample_movie_indices = []
for decade in decades:
    for i in range(450):
        index = random.randint(0, (len(movie_decades)-1))
        while (movie_decades[index] != decade) | (index in sample_movie_indices):
            index = random.randint(0, (len(movie_decades)-1))
        sample_movie_indices.append(index)
        

In [9]:
print(len(sample_movie_indices))
print(len(movie_decades))

4050
13430


In [10]:
movievectors = []
sample_metadata = []

for i in range(len(sample_movie_indices)):
    index = sample_movie_indices[i]
    movie = allmovies[index]
    totvec = np.zeros(100)
    for line in movie:
        for token in line.split(" "):
            try:
                totvec = totvec + model.wv[token]
            except KeyError:
                continue
    if (i%100 == 0):
        print(str(i))
   
    movievectors.append(totvec)
    
    id = movie_ids[index]
    title = movie_titles[index]
    decade = movie_decades[index]
    year = movie_years[index]
    genre = movie_genres[index]
    meta = [index, id, title, year, decade, genre]
    sample_metadata.append(meta)
    

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000


TypeError: to_excel() got an unexpected keyword argument 'sheetname'

In [11]:
sample_df = pd.DataFrame(sample_metadata, columns=['index','textID','title','year','decade','genre'])
sample_df.to_excel("sample_movie_metadata.xlsx", sheet_name='Sample', index=False)

In [16]:
with open('movie_vectors', 'w') as f:
    for vector in movievectors:
        f.write(str(vector) + '\n')

In [12]:
from sklearn.cluster import KMeans

kmmovies = KMeans(n_clusters=20, random_state=0)
movieclusters = kmmovies.fit_predict(movievectors)

In [22]:
for i in range(len(movieclusters)):
    if movieclusters[i] == 4:
        print(sample_metadata[i])

[160, 24963, 'Chained', 1934, '1930s', 'Drama, Romance']
[206, 26073, 'Annie Oakley', 1935, '1930s', 'Biography, Drama, Western']
[384, 30865, 'Three Comrades', 1938, '1930s', 'Drama, Romance']
[25, 21730, 'The Champ', 1931, '1930s', 'Drama, Sport']
[113, 23940, 'Design for Living', 1933, '1930s', 'Comedy, Romance']
[447, 32145, 'Wuthering Heights', 1939, '1930s', 'Drama, Romance']
[336, 29345, 'On the Avenue', 1937, '1930s', 'Musical, Comedy, Romance']
[394, 31088, 'Beau Geste', 1939, '1930s', 'Action, Adventure, Drama']
[290, 28138, 'The Princess Comes Across', 1936, '1930s', 'Comedy, Mystery, Romance']
[238, 26778, 'A Night at the Opera', 1935, '1930s', 'Comedy, Music, Musical']
[189, 25617, 'The Painted Veil', 1934, '1930s', 'Drama, Romance']
[42, 22276, 'Possessed', 1931, '1930s', 'Drama, Romance']
[89, 23385, 'Red-Headed Woman', 1932, '1930s', 'Comedy, Romance']
[85, 23335, 'The Phantom of Crestwood', 1932, '1930s', 'Crime, Mystery']
[58, 22735, 'The Cabin in the Cotton', 1932, '

In [23]:
with open('clusters', 'w') as f:
    for i in range(len(movieclusters)):
        f.write("\nCLUSTER {num}\n".format(num=i))
        for j in range(len(movieclusters)):
            if movieclusters[j] == i:
                f.write(str(sample_metadata[j]) + "\n")